In [1]:
import warnings
import numpy as np
import osmnx as ox
import pandas as pd
import geopandas as gpd
import plotly.express as px
# from skgstat import Variogram
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon
# from pykrige.ok import OrdinaryKriging
from scipy.interpolate import NearestNDInterpolator
from tobler.area_weighted import area_interpolate
import itertools

# Plotting defaults
plt.style.use('ggplot')
px.defaults.height = 400; px.defaults.width = 620
plt.rcParams.update({'font.size': 16, 'axes.labelweight': 'bold', 'figure.figsize': (6, 6), 'axes.grid': False})

/opt/conda/lib/python3.7/site-packages/geopandas/_compat.py:115: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [2]:
import os
import torch
import torchvision
from torch import nn, optim
from torchvision import transforms, models, datasets
from torchsummary import summary
from PIL import Image
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

ModuleNotFoundError: No module named 'torchsummary'

In [ ]:
def pixel2poly(x, y, z, resolution):
    """
    x: x coords of cell
    y: y coords of cell
    z: matrix of values for each (x,y)
    resolution: spatial resolution of each cell
    """
    polygons = []
    values = []
    half_res = resolution / 2
    for i, j  in itertools.product(range(len(x)), range(len(y))):
        minx, maxx = x[i] - half_res, x[i] + half_res
        miny, maxy = y[j] - half_res, y[j] + half_res
        polygons.append(Polygon([(minx, miny), (minx, maxy), (maxx, maxy), (maxx, miny)]))
        if isinstance(z, (int, float)):
            values.append(z)
        else:
            values.append(z[j, i])
    return polygons, values

In [ ]:
train = pd.read_csv('/kaggle/input/bizinnovate-2023/train.csv')
test = pd.read_csv('/kaggle/input/bizinnovate-2023/test_masked.csv')
train.head()

In [ ]:
corr = train.corr()
corr.style.background_gradient(cmap='coolwarm')

In [ ]:
train[['water_index', 'asset_index']].hist(figsize=(10,6))
plt.show()

In [ ]:
ex = np.load('/kaggle/input/bizinnovate-2023/dhs_train/IA-2015-7-00010004.npz')
ex.files

In [ ]:
ex['x'].shape

In [ ]:
fig, axs = plt.subplots(2, 4, figsize=(15,8))

axs = axs.ravel()

for i in range(8):

    axs[i].contourf(ex['x'][i])

In [ ]:
train.loc[train['DHSID_EA'] == 'IA-2015-7-00010004', 'water_index'].values[0]

In [ ]:
ex2 = np.load('/kaggle/input/bizinnovate-2023/dhs_train/IA-2015-7-00010005.npz')

fig, axs = plt.subplots(2, 4, figsize=(15,8))

axs = axs.ravel()

for i in range(8):

    axs[i].contourf(ex2['x'][i])

In [ ]:
train.loc[train['DHSID_EA'] == 'IA-2015-7-00010005', 'water_index'].values[0]

In [ ]:
# top 3 max water_index, asset_index
train.sort_values(by=['water_index', 'asset_index'], ascending=False).iloc[:3]

In [ ]:
# top 3 min water_index, asset_index
train.sort_values(by=['water_index', 'asset_index'], ascending=True).iloc[:3]

In [ ]:
best_ex1 = np.load('/kaggle/input/bizinnovate-2023/dhs_train/IA-2015-7-00110590.npz')
best_ex2 = np.load('/kaggle/input/bizinnovate-2023/dhs_train/IA-2015-7-00270038.npz')
best_ex3 = np.load('/kaggle/input/bizinnovate-2023/dhs_train/IA-2015-7-00270084.npz')

fig1, axs1 = plt.subplots(2, 4, figsize=(15,8))
fig2, axs2 = plt.subplots(2, 4, figsize=(15,8))
fig3, axs3 = plt.subplots(2, 4, figsize=(15,8))

axs1 = axs1.ravel()
axs2 = axs2.ravel()
axs3 = axs3.ravel()

for i in range(8):
    axs1[i].contourf(best_ex1['x'][i])
    axs2[i].contourf(best_ex2['x'][i])
    axs3[i].contourf(best_ex3['x'][i])
fig1.suptitle('Max Water Index');

In [ ]:
worst_ex1 = np.load('/kaggle/input/bizinnovate-2023/dhs_train/IA-2015-7-00041022.npz')
worst_ex2 = np.load('/kaggle/input/bizinnovate-2023/dhs_train/IA-2015-7-00040712.npz')
worst_ex3 = np.load('/kaggle/input/bizinnovate-2023/dhs_train/IA-2015-7-00030236.npz')

fig1, axs1 = plt.subplots(2, 4, figsize=(15,8))
fig2, axs2 = plt.subplots(2, 4, figsize=(15,8))
fig3, axs3 = plt.subplots(2, 4, figsize=(15,8))

axs1 = axs1.ravel()
axs2 = axs2.ravel()
axs3 = axs3.ravel()

for i in range(8):
    axs1[i].contourf(worst_ex1['x'][i])
    axs2[i].contourf(worst_ex2['x'][i])
    axs3[i].contourf(worst_ex3['x'][i])
fig1.suptitle('Min Water Index');

In [ ]:
print(best_ex1['x'].mean(axis=(1,2)))
print(best_ex2['x'].mean(axis=(1,2)))
print(best_ex3['x'].mean(axis=(1,2)))

In [ ]:
print(worst_ex1['x'].mean(axis=(1,2)))
print(worst_ex2['x'].mean(axis=(1,2)))
print(worst_ex3['x'].mean(axis=(1,2)))

In [ ]:
keys = ['mean_energy_0', 'mean_energy_1', 'mean_energy_2',
        'mean_energy_3', 'mean_energy_4', 'mean_energy_5',
        'mean_energy_6', 'mean_energy_7']
dict(zip(keys, worst_ex3['x'].mean(axis=(1,2))))

In [ ]:
df_keys = keys + ['DHSID_EA']
init_dict = dict(zip(df_keys, [[],[],[],[],[],[],[],[],[]]))
energy_df = pd.DataFrame(init_dict)
energy_df

In [ ]:
# Feature engineering
for i, row in train.iterrows():
    # Load
    data = np.load(f"/kaggle/input/bizinnovate-2023/dhs_train/{str(row['DHSID_EA'])}.npz")
    # Extract & transform
    insert = dict(zip(keys, data['x'].mean(axis=(1,2))))
    insert['DHSID_EA'] = str(row['DHSID_EA'])
    energy_df = pd.concat([energy_df, pd.DataFrame([insert])])

In [ ]:
energy_df

In [ ]:
energy_df.to_csv('train_energy.csv')

In [ ]:
corr = energy_df.corr()
corr.style.background_gradient(cmap='coolwarm')

In [ ]:
train_eng = pd.merge(train, energy_df, on='DHSID_EA', how='left')
train_eng.head()

In [ ]:
train_feats = train_eng[['lat', 'lon', 'asset_index', 'water_index',
                         'mean_energy_0', 'mean_energy_1', 'mean_energy_2',
                         'mean_energy_3', 'mean_energy_4', 'mean_energy_5',
                         'mean_energy_6', 'mean_energy_7']]
train_feats.head()

In [ ]:
corr = train_feats.corr()
corr.style.background_gradient(cmap='coolwarm')

In [ ]:
df_keys = keys + ['DHSID_EA']
init_dict = dict(zip(df_keys, [[],[],[],[],[],[],[],[],[]]))
test_energy_df = pd.DataFrame(init_dict)
test_energy_df

In [ ]:
for i, row in test.iterrows():
    # Load
    data = np.load(f"/kaggle/input/bizinnovate-2023/dhs_test/{str(row['DHSID_EA'])}.npz")
    # Extract & transform
    insert = dict(zip(keys, data['x'].mean(axis=(1,2))))
    insert['DHSID_EA'] = str(row['DHSID_EA'])
    test_energy_df = pd.concat([test_energy_df, pd.DataFrame([insert])])

In [ ]:
energy_df.to_csv('test_energy.csv')

In [ ]:
test_eng = pd.merge(test, test_energy_df, on='DHSID_EA', how='left')
test_eng.head()

In [ ]:
test_feats = test_eng[['lat', 'lon', 'asset_index',
                       'mean_energy_0', 'mean_energy_1', 'mean_energy_2',
                       'mean_energy_3', 'mean_energy_4', 'mean_energy_5',
                       'mean_energy_6', 'mean_energy_7']]
test_feats.head()

In [ ]:
train_energy_df = pd.read_csv('/kaggle/working/train_energy.csv')
train_eng = pd.merge(test, train_energy_df, on='DHSID_EA', how='left')

train_feats = train_eng[['lat', 'lon', 'asset_index', 'water_index',
                         'mean_energy_0', 'mean_energy_1', 'mean_energy_2',
                         'mean_energy_3', 'mean_energy_4', 'mean_energy_5',
                         'mean_energy_6', 'mean_energy_7']]
train_feats.head()

In [ ]:
from sklearn.model_selection  import train_test_split, cross_val_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

X = train_eng[['lat', 'lon', 'asset_index', 'mean_energy_5', 'mean_energy_6', 'mean_energy_7']]
y = train_eng['water_index']

Xt = test_eng[['lat', 'lon', 'asset_index', 'mean_energy_5', 'mean_energy_6', 'mean_energy_7']]

X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.33, random_state=42)

In [ ]:
scores = []
for k in np.arange(1, 30, 2):   # running for different K values to know which yields the max accuracy. 
    clf = KNeighborsRegressor(n_neighbors = k, weights='uniform', algorithm="auto")
    knn_pipe = make_pipeline(StandardScaler(), clf)
    knn_pipe.fit(X_train, y_train)
    score = cross_val_score(knn_pipe, X_train, y_train, cv = 10)
    scores.append(score.mean())

In [ ]:
pd.DataFrame(scores, index=list(np.arange(1, 30, 2)))

In [ ]:
knn = make_pipeline(StandardScaler(), KNeighborsRegressor(n_neighbors = 17, weights='uniform', algorithm="auto"))
knn.fit(X_train, y_train)
y_valid_pred = knn.predict(X_valid)

In [ ]:
r2_score(y_valid, y_valid_pred)

In [ ]:
from lightgbm import LGBMRegressor

lgbm_pipe = make_pipeline(StandardScaler(), LGBMRegressor())
lgbm_pipe.fit(X_train, y_train)
predicted_y = lgbm_pipe.predict(X_valid)

In [ ]:
r2_score(y_valid, predicted_y)

In [ ]:
import xgboost as xgb

xgb_pipe = make_pipeline(StandardScaler(), xgb.XGBRegressor(tree_method="hist", eval_metric=r2_score))
xgb_pipe.fit(X_train, y_train)
predicted_y = reg.predict(X_valid)

In [ ]:
r2_score(y_valid, predicted_y)

In [ ]:
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform
from sklearn.model_selection import RandomizedSearchCV

param_test = {'lgbmregressor__learning_rate' : [0.01, 0.02, 0.03, 0.04, 0.05, 0.08, 0.1, 0.2, 0.3, 0.4],
              'lgbmregressor__n_estimators' : [100, 200, 300, 400, 500, 600, 800, 1000, 1500, 2000],
              'lgbmregressor__num_leaves': sp_randint(6, 50), 
              'lgbmregressor__min_child_samples': sp_randint(100, 500), 
              'lgbmregressor__min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4],
              'lgbmregressor__subsample': sp_uniform(loc = 0.2, scale = 0.8), 
              'lgbmregressor__max_depth': [-1, 1, 2, 3, 4, 5, 6, 7],
              'lgbmregressor__colsample_bytree': sp_uniform(loc = 0.4, scale = 0.6),
              'lgbmregressor__reg_alpha': [0, 1e-1, 1, 2, 5, 7, 10, 50, 100],
              'lgbmregressor__reg_lambda': [0, 1e-1, 1, 5, 10, 20, 50, 100]}

lgbm_clf = LGBMRegressor(random_state = 42, metric = 'r2', n_jobs = 4)
lgbm_clf_pipe = make_pipeline(StandardScaler(), lgbm_clf)
grid_search = RandomizedSearchCV(
    estimator = lgbm_clf_pipe, param_distributions = param_test, 
    n_iter = 500,
    scoring = 'r2',
    cv = 5,
    refit = True,
    random_state = 42,
    verbose = True)

grid_search.fit(X_train, y_train)
opt_parameters = grid_search.best_params_

In [ ]:
lgbm_clf = lgbm.LGBMClassifier(**opt_parameters)
y_pred = lgbm_clf.fit(X_train, y_train).predict(X_valid)

In [ ]:
r2_score(y_valid, y_pred)

In [ ]:
preds = lgbm.predict(Xt)
test['water_index'] = preds
tester = test[['DHSID_EA', 'water_index']]
tester.to_csv('test.csv', index=False)

In [ ]:
train['path'] = train['path'].str.replace('dhs_train/', '')
img_train_dict = dict(zip(train['path'], train['water_index']))

In [ ]:
train_dir = '/kaggle/input/bizinnovate-2023/dhs_train'

img_shape = (8, 255, 255)

img_names = np.sort(os.listdir(train_dir))

In [ ]:
img_names

In [ ]:
img_train_dict['IA-2015-7-00010004.npz']

In [ ]:
geotrain = (gpd.GeoDataFrame(
                train, 
                crs="EPSG:4326", # specify the CRS
                geometry=gpd.points_from_xy(train["lon"], train["lat"])) # create geometry from coordinates
            .to_crs("EPSG:3347") # convert to a different CRS
            )
geotrain["Easting"], geotrain["Northing"] = geotrain.geometry.x, geotrain.geometry.y
geotrain.head()

In [ ]:
resolution = 10000  # cell size in meters, smaller cell size = smaller pixel = higher resolution 
gridx = np.arange(geotrain.bounds.minx.min(), geotrain.bounds.maxx.max(), resolution)
gridy = np.arange(geotrain.bounds.miny.min(), geotrain.bounds.maxy.max(), resolution)

In [ ]:
model = NearestNDInterpolator(x = list(zip(geotrain["Easting"], geotrain["Northing"])),
                              y = geotrain["water_index"])
z = model(*np.meshgrid(gridx, gridy))
plt.imshow(z);

In [ ]:
polygons, values = pixel2poly(gridx, gridy, z, resolution)

In [ ]:
geotrain_model = (gpd.GeoDataFrame({"water_index_modelled": values}, geometry=polygons, crs="EPSG:3347")
                      .to_crs("EPSG:4326")
                 )

fig = px.choropleth_mapbox(geotrain_model, geojson=geotrain_model.geometry, locations=geotrain_model.index,
                           color="water_index_modelled", opacity=0.5,
#                            center={"lat": 52.261, "lon": -123.246}, zoom=3.5,
                           mapbox_style="carto-darkmatter")
fig.update_layout(margin=dict(l=0, r=0, t=30, b=10))
fig.update_traces(marker_line_width=0)

In [ ]:
geotest = (gpd.GeoDataFrame(
            test, 
            crs="EPSG:4326", # specify the CRS
            geometry=gpd.points_from_xy(test["lon"], test["lat"])) # create geometry from coordinates
          .to_crs("EPSG:3347") # convert to a different CRS
            )
geotest["Easting"], geotest["Northing"] = geotest.geometry.x, geotest.geometry.y
geotest.head()

In [ ]:
resolution = 500  # cell size in meters, smaller cell size = smaller pixel = higher resolution 
gridx = np.arange(geotest.bounds.minx.min(), geotest.bounds.maxx.max(), resolution)
gridy = np.arange(geotest.bounds.miny.min(), geotest.bounds.maxy.max(), resolution)

In [ ]:
model = NearestNDInterpolator(x = list(zip(geotrain["Easting"], geotrain["Northing"])),
                              y = geotrain["water_index"], rescale=True, tree_options={'leafsize': 20})
# z = model(*np.meshgrid(gridx, gridy))
# plt.imshow(z);

In [ ]:
geotest['water_index'] = geotest.apply(lambda x: model(x.Easting, x.Northing), axis=1)

In [ ]:
preds = geotest[['DHSID_EA', 'water_index']]

In [ ]:
preds.to_csv('test.csv', index=False)

In [ ]:
preds.shape

In [ ]:
preds

In [ ]:
pd.read_csv('/kaggle/working/test.csv')

In [ ]:
train

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

X = train[['lat', 'lon']]
y = train['water_index']

Xt = test[['lat', 'lon']]

neigh = KNeighborsRegressor(n_neighbors=2)
neigh.fit(X, y)
preds = neigh.predict(X)

In [ ]:
geotest['water_index'] = preds

In [ ]:
preds = geotest[['DHSID_EA', 'water_index']]

In [ ]:
preds

In [ ]:
from sklearn.model_selection  import train_test_split, cross_val_score
from sklearn.neighbors import KNeighborsRegressor

X = train[['lat', 'lon', 'asset_index']]
y = train['water_index']

Xt = test[['lat', 'lon', 'asset_index']]

X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.33, random_state=42)

scores = []
for k in np.arange(1, 30, 2):   # running for different K values to know which yields the max accuracy. 
    clf = KNeighborsRegressor(n_neighbors = k, weights='uniform', algorithm="auto")
    clf.fit(X_train, y_train)
    score = cross_val_score(clf, X_train, y_train, cv = 10)
    scores.append(score.mean())

In [ ]:
pd.DataFrame(scores, index=list(np.arange(1, 30, 2)))

In [ ]:
knn = KNeighborsRegressor(n_neighbors = 9, weights='uniform', algorithm="auto")
knn.fit(X_train, y_train)
y_valid_pred = knn.predict(X_valid)

In [ ]:
from sklearn.metrics import r2_score
r2_score(y_valid, y_valid_pred)

In [ ]:
preds = knn.predict(Xt)
test['water_index'] = preds
test = test[['DHSID_EA', 'water_index']]
test.to_csv('test.csv', index=False)

In [ ]:
from lightgbm import LGBMRegressor

lgbm = LGBMRegressor()
lgbm.fit(X_train, y_train)
predicted_y = lgbm.predict(X_valid)

In [ ]:
r2_score(y_valid, predicted_y)

In [ ]:
preds = lgbm.predict(Xt)
test['water_index'] = preds
tester = test[['DHSID_EA', 'water_index']]
tester.to_csv('test.csv', index=False)

In [ ]:
import xgboost as xgb

reg = xgb.XGBRegressor(
    tree_method="hist",
    eval_metric=r2_score,
)
reg.fit(X_train, y_train)
predicted_y = reg.predict(X_valid)

In [ ]:
r2_score(y_valid, predicted_y)

In [ ]:
krig = OrdinaryKriging(x=geotrain["Easting"], y=geotrain["Northing"], z=geotrain["water_index"], variogram_model="spherical")

In [ ]:
preds, ss = krig.execute("points", geotest['Easting'], geotest['Northing'])

In [ ]:
geotest['water_index'] = preds

In [ ]:
preds = geotest[['DHSID_EA', 'water_index']]

In [ ]:
preds

In [ ]:
preds.to_csv('test.csv', index=False)

In [ ]:
preds10, ss10 = krig.execute("points", geotrain['Easting'][:10], geotrain['Northing'][:10])

In [ ]:
from sklearn.metrics import r2_score

r2_score(y[:10], preds10)